In [17]:
import numpy as np
import gpkit
from gpkit import Model, Variable

In [18]:
#define a module describing the water
class Material():
    def __init__(self, name, rho, sigma_yield, sigma_ultimate):
        self.name = name
        self.rho  = rho
        self.sigma_yield = sigma_yield
        self.sigma_ultimate = sigma_ultimate
        
    def __str__(self):
        string = f"Material: {self.name}"
        
        return string


class Water(Model):
    def setup(self):
        rho = Variable("\\rho", 1000, "kg/m^3", "Water density")
        m   = self.m = Variable("m", "g", "Mass of water")
        V   = self.V = Variable("V", "L", "Volume of water")
        
        V_min = 0.35*gpkit.ureg.L
        
        return [m == rho*V, V >= V_min]
    
class GeneralTank(Model):
    def setup(self, P, material):
        
        #assumes the P is the maximum operating pressure 
        constraints = []
        
        
        #design
        r = self.r = Variable("r", "cm", "Tank Radius")
        t = self.t = Variable("t", "mm", "Tank Thickness")
        l = self.l = Variable("l", "cm", "Tank Length")
        
        self.V_internal = np.pi*r**2*l
        constraints += [r >= 10*t] #ensures thin wall approx can be used. 
        
        #pressures
        FOSY = Variable("FOSY", 1.0, "", "Factor of Safety: Yield")
        FOSU = Variable("FOSU", 1.0, "", "Factor of Safety: Ultimate")
        jb   = Variable("j_{burst}", 1.5, "", "burst factor")
        jp   = Variable("j_{proof}", 1.25, "", "proof factor")
        
        P_burst = self.P_burst = Variable("P_{burst}", "bar", "burst pressure")
        P_proof = self.P_proof = Variable("P_{proof}", "bar", "proof pressure")
        
        
        constraints += [P_burst >= jb*(1+FOSU)*P]
        constraints += [P_proof >= jp*(1+FOSY)*P]
        
        #stresses
        sigma_vm_p = self.sigma_vm_p = Variable("\\sigma_{vm, proof}", "MPa", "Proof Von misses stress in tank")
        sigma_vm_b = self.sigma_vm_b = Variable("\\sigma_{vm, burst}", "MPa", "Burst Von misses stress in tank")
        
        constraints += [sigma_vm_p == (np.sqrt(3)/2)*P_proof*r/t]
        constraints += [sigma_vm_b == (np.sqrt(3)/2)*P_burst*r/t]
        
        constraints += [sigma_vm_p <= material.sigma_yield]
        constraints += [sigma_vm_b <= material.sigma_ultimate]
        
        #mass
        m = self.m = Variable("m", "g", "Mass of tank")
        
        rho = self.rho = material.rho
        
        #mass calculator
        constraints += [m>= rho*(2*np.pi*r*t*l + 2*np.pi*r**2*t)]
        
        return constraints

rho_al = Variable("\\rho_{Al}", 2.7,"g/cm^3","Al tank density")
sigma_yield = Variable("\\sigma_{y, al}", 276, "MPa", "Al yield strength")
sigma_ultimate = Variable("\\sigma_{u, al}", 310, "MPa", "Al ultimate strength")
al = Material('Al', rho=rho_al, sigma_yield=sigma_yield, sigma_ultimate = sigma_ultimate)


class MainTank(Model):
    def setup(self):
        
        #define operation condition
        P_expected = self.P_expected = Variable("P_{expected}", 4, "bar", "Maximum expected pressure")
        
        #define tank material
        self.material = al
        
        #create tank
        tank = self.tank = GeneralTank(P_expected, material = self.material)
        
        #create water
        water = Water()
        #assemble components
        components = self.components = [water, tank]
        
        constraints = []        
        constraints += components
        
        #require water to fit in tank
        constraints += [water.V <= tank.V_internal]
        #require tank to be smaller than 1U
        constraints += [tank.l <= 10*gpkit.ureg.cm]
        constraints += [tank.r <=  5*gpkit.ureg.cm]
        
        
        #determine total mass
        m = self.m = Variable("m", "g", " Main Tank Mass")
        
        constraints += [m >= sum(comp.m for comp in components)]
        
        return constraints
    
    
class Helium(Model):
    def setup(self, P, V, m, T):
        #describes the ideal gas law of helium
        constraints = []
        #gas constants
        R = 8.31446261815324*(gpkit.ureg.J/gpkit.ureg.K)/gpkit.ureg.mol
        M = 4.002602*gpkit.ureg.g/gpkit.ureg.mol
        
        self.m = m
        
        constraints += [P*V == m*R*T/M]
        
        return constraints
    
class HeliumTank(Model):
    def setup(self):
        
        constraints = []
        components = self.components = []
        
        #create tank
        P_He_i = self.P_He_i = Variable("P_{He, initial}", "bar", "Initial maximum expected pressure of Helium Tank")
        
        tank = self.tank=GeneralTank(P_He_i, material = al)
        components += [tank]
        
        #create helium
        m_He = self.m_He = Variable("m_{He}", "g", "Mass of Helium")
        T = self.T = Variable("T", "K", "Helium Temperature")
        
        constraints += [m_He >= 1*gpkit.ureg.g] #define minimum mass of Helium
        constraints += [T >= 200*gpkit.ureg.K, T <= 300*gpkit.ureg.K]
        
        He_initial = Helium(P = P_He_i, V = tank.V_internal, m = m_He, T = T)
        
        self.He_initial = He_initial

        components += [He_initial]
        
        #determine total mass
        m = self.m = Variable("m", "g", " Main Tank Mass")
        
        constraints += [m >= sum(comp.m for comp in components)]
        
        constraints += components
        
        return constraints
        
        

In [26]:
class TankSystem(Model):
    def setup(self):
        
        constraints = []
        
        mainTank = MainTank()
        heliumTank = HeliumTank()
        
        components = self.components = [mainTank, heliumTank]
        constraints += [components]
        
        #create final helium, but dont include in mass breakdown
        He_final = Helium(P = mainTank.P_expected, V = mainTank.tank.V_internal, m = heliumTank.m_He, T = heliumTank.T)
        
        constraints += [He_final]
        #constraints += [He_initial.m >= 1*gpkit.ureg.g]

        
        m = self.m = Variable("m", "g", "mass of full tank system")
        constraints += [m >= sum(comp.m for comp in components)]
        
        return constraints

In [27]:
ht = HeliumTank()
prob = Model(ht.m, ht)

In [28]:
prob

[[gpkit.PosynomialInequality(m_{He}_HeliumTank.3 >= 1),
  gpkit.PosynomialInequality(T_HeliumTank.3 >= 200),
  gpkit.PosynomialInequality(T_HeliumTank.3 <= 300),
  gpkit.PosynomialInequality(m_HeliumTank.3 >= m_HeliumTank.3/GeneralTank + m_{He}_HeliumTank.3),
  [gpkit.PosynomialInequality(r_HeliumTank.3/GeneralTank >= 10*t_HeliumTank.3/GeneralTank),
   gpkit.PosynomialInequality(P_{burst}_HeliumTank.3/GeneralTank >= FOSU_HeliumTank.3/GeneralTank*P_{He, initial}_HeliumTank.3*j_{burst}_HeliumTank.3/GeneralTank + P_{He, initial}_HeliumTank.3*j_{burst}_HeliumTank.3/GeneralTank),
   gpkit.PosynomialInequality(P_{proof}_HeliumTank.3/GeneralTank >= FOSY_HeliumTank.3/GeneralTank*P_{He, initial}_HeliumTank.3*j_{proof}_HeliumTank.3/GeneralTank + P_{He, initial}_HeliumTank.3*j_{proof}_HeliumTank.3/GeneralTank),
   gpkit.MonomialEquality(\sigma_{vm, proof}_HeliumTank.3/GeneralTank = 0.866*P_{proof}_HeliumTank.3/GeneralTank*r_HeliumTank.3/GeneralTank*t_HeliumTank.3/GeneralTank**-1),
   gpkit.Monomi

In [29]:
prob.solve()

ValueError: Geometric Program is not fully bounded:
  \sigma_{y, al} has no upper bound
  \sigma_{u, al} has no upper bound
  \rho_{Al} has no lower bound

In [30]:
print(Model(ht.m, [ht, ht.He_initial.m>=1*gpkit.ureg.g]).solve(verbosity=0).table())

ValueError: Geometric Program is not fully bounded:
  \sigma_{y, al} has no upper bound
  \sigma_{u, al} has no upper bound
  \rho_{Al} has no lower bound

In [31]:
ts=TankSystem()

In [32]:
problem = Model(ts.m, ts)
problem

[[[[[gpkit.MonomialEquality(m_TankSystem.1/MainTank/Water = V_TankSystem.1/MainTank/Water*\rho_TankSystem.1/MainTank/Water),
     gpkit.PosynomialInequality(V_TankSystem.1/MainTank/Water >= 0.35)],
    [gpkit.PosynomialInequality(r_TankSystem.1/MainTank/GeneralTank >= 10*t_TankSystem.1/MainTank/GeneralTank),
     gpkit.PosynomialInequality(P_{burst}_TankSystem.1/MainTank/GeneralTank >= FOSU_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank),
     gpkit.PosynomialInequality(P_{proof}_TankSystem.1/MainTank/GeneralTank >= FOSY_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank),
     gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/MainTank/GeneralTa

In [33]:
ts

[[[[gpkit.MonomialEquality(m_TankSystem.1/MainTank/Water = V_TankSystem.1/MainTank/Water*\rho_TankSystem.1/MainTank/Water),
    gpkit.PosynomialInequality(V_TankSystem.1/MainTank/Water >= 0.35)],
   [gpkit.PosynomialInequality(r_TankSystem.1/MainTank/GeneralTank >= 10*t_TankSystem.1/MainTank/GeneralTank),
    gpkit.PosynomialInequality(P_{burst}_TankSystem.1/MainTank/GeneralTank >= FOSU_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank),
    gpkit.PosynomialInequality(P_{proof}_TankSystem.1/MainTank/GeneralTank >= FOSY_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank),
    gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/MainTank/GeneralTank*r_T

In [34]:
ts.components[0]

[[gpkit.MonomialEquality(m_TankSystem.1/MainTank/Water = V_TankSystem.1/MainTank/Water*\rho_TankSystem.1/MainTank/Water),
  gpkit.PosynomialInequality(V_TankSystem.1/MainTank/Water >= 0.35)],
 [gpkit.PosynomialInequality(r_TankSystem.1/MainTank/GeneralTank >= 10*t_TankSystem.1/MainTank/GeneralTank),
  gpkit.PosynomialInequality(P_{burst}_TankSystem.1/MainTank/GeneralTank >= FOSU_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank),
  gpkit.PosynomialInequality(P_{proof}_TankSystem.1/MainTank/GeneralTank >= FOSY_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank),
  gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/MainTank/GeneralTank*r_TankSystem.1/

In [35]:
ts.components[1]

[gpkit.PosynomialInequality(m_{He}_TankSystem.1/HeliumTank >= 1),
 gpkit.PosynomialInequality(T_TankSystem.1/HeliumTank >= 200),
 gpkit.PosynomialInequality(T_TankSystem.1/HeliumTank <= 300),
 gpkit.PosynomialInequality(m_TankSystem.1/HeliumTank >= m_TankSystem.1/HeliumTank/GeneralTank + m_{He}_TankSystem.1/HeliumTank),
 [gpkit.PosynomialInequality(r_TankSystem.1/HeliumTank/GeneralTank >= 10*t_TankSystem.1/HeliumTank/GeneralTank),
  gpkit.PosynomialInequality(P_{burst}_TankSystem.1/HeliumTank/GeneralTank >= FOSU_TankSystem.1/HeliumTank/GeneralTank*P_{He, initial}_TankSystem.1/HeliumTank*j_{burst}_TankSystem.1/HeliumTank/GeneralTank + P_{He, initial}_TankSystem.1/HeliumTank*j_{burst}_TankSystem.1/HeliumTank/GeneralTank),
  gpkit.PosynomialInequality(P_{proof}_TankSystem.1/HeliumTank/GeneralTank >= FOSY_TankSystem.1/HeliumTank/GeneralTank*P_{He, initial}_TankSystem.1/HeliumTank*j_{proof}_TankSystem.1/HeliumTank/GeneralTank + P_{He, initial}_TankSystem.1/HeliumTank*j_{proof}_TankSystem.1/

In [36]:
ts.components[0].components[0]

[gpkit.MonomialEquality(m_TankSystem.1/MainTank/Water = V_TankSystem.1/MainTank/Water*\rho_TankSystem.1/MainTank/Water),
 gpkit.PosynomialInequality(V_TankSystem.1/MainTank/Water >= 0.35)]

In [37]:
ts.components[0].components[1]

[gpkit.PosynomialInequality(r_TankSystem.1/MainTank/GeneralTank >= 10*t_TankSystem.1/MainTank/GeneralTank),
 gpkit.PosynomialInequality(P_{burst}_TankSystem.1/MainTank/GeneralTank >= FOSU_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank),
 gpkit.PosynomialInequality(P_{proof}_TankSystem.1/MainTank/GeneralTank >= FOSY_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank),
 gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/MainTank/GeneralTank*r_TankSystem.1/MainTank/GeneralTank*t_TankSystem.1/MainTank/GeneralTank**-1),
 gpkit.MonomialEquality(\sigma_{vm, burst}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{burst}_TankSystem.1/MainTank/GeneralTank*r_Ta

In [38]:
ts.components[1].components[0]

[gpkit.PosynomialInequality(r_TankSystem.1/HeliumTank/GeneralTank >= 10*t_TankSystem.1/HeliumTank/GeneralTank),
 gpkit.PosynomialInequality(P_{burst}_TankSystem.1/HeliumTank/GeneralTank >= FOSU_TankSystem.1/HeliumTank/GeneralTank*P_{He, initial}_TankSystem.1/HeliumTank*j_{burst}_TankSystem.1/HeliumTank/GeneralTank + P_{He, initial}_TankSystem.1/HeliumTank*j_{burst}_TankSystem.1/HeliumTank/GeneralTank),
 gpkit.PosynomialInequality(P_{proof}_TankSystem.1/HeliumTank/GeneralTank >= FOSY_TankSystem.1/HeliumTank/GeneralTank*P_{He, initial}_TankSystem.1/HeliumTank*j_{proof}_TankSystem.1/HeliumTank/GeneralTank + P_{He, initial}_TankSystem.1/HeliumTank*j_{proof}_TankSystem.1/HeliumTank/GeneralTank),
 gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/HeliumTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/HeliumTank/GeneralTank*r_TankSystem.1/HeliumTank/GeneralTank*t_TankSystem.1/HeliumTank/GeneralTank**-1),
 gpkit.MonomialEquality(\sigma_{vm, burst}_TankSystem.1/HeliumTank/GeneralTank = 0.86

In [39]:
ts.components[1].components[1]

[gpkit.MonomialEquality(3.14*P_{He, initial}_TankSystem.1/HeliumTank*l_TankSystem.1/HeliumTank/GeneralTank*r_TankSystem.1/HeliumTank/GeneralTank**2 = 2.08*T_TankSystem.1/HeliumTank*m_{He}_TankSystem.1/HeliumTank)]

In [40]:
problem

[[[[[gpkit.MonomialEquality(m_TankSystem.1/MainTank/Water = V_TankSystem.1/MainTank/Water*\rho_TankSystem.1/MainTank/Water),
     gpkit.PosynomialInequality(V_TankSystem.1/MainTank/Water >= 0.35)],
    [gpkit.PosynomialInequality(r_TankSystem.1/MainTank/GeneralTank >= 10*t_TankSystem.1/MainTank/GeneralTank),
     gpkit.PosynomialInequality(P_{burst}_TankSystem.1/MainTank/GeneralTank >= FOSU_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{burst}_TankSystem.1/MainTank/GeneralTank),
     gpkit.PosynomialInequality(P_{proof}_TankSystem.1/MainTank/GeneralTank >= FOSY_TankSystem.1/MainTank/GeneralTank*P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank + P_{expected}_TankSystem.1/MainTank*j_{proof}_TankSystem.1/MainTank/GeneralTank),
     gpkit.MonomialEquality(\sigma_{vm, proof}_TankSystem.1/MainTank/GeneralTank = 0.866*P_{proof}_TankSystem.1/MainTank/GeneralTa

In [41]:
print(problem.solve(verbosity=0).table())

ValueError: Geometric Program is not fully bounded:
  \sigma_{y, al} has no upper bound
  \sigma_{u, al} has no upper bound
  \rho_{Al} has no lower bound

In [42]:
problem.solve(verbosity=0)

ValueError: Geometric Program is not fully bounded:
  \sigma_{y, al} has no upper bound
  \sigma_{u, al} has no upper bound
  \rho_{Al} has no lower bound

In [44]:
problem.debug()

< DEBUGGING >
> Trying with bounded variables and relaxed constants:
>> Failure.
> Trying with relaxed constraints:


IndexError: index 1 is out of bounds for axis 0 with size 1